##### Importing 

In [1]:
import cv2
import numpy as np
import mediapipe as mp

##### Practice

In [ ]:
# img = np.zeros((400,700))

In [ ]:
# data = cv2.line(img,(300,350),(350,300),(255,255,255),2)

In [ ]:
# cv2.imshow("Image",img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

##### Main Code

Video Frame size uisng set()</br>
</br>
3 (or) cv2.CAP_PROP_FRAME_WIDTH	  =>   Sets the width of the video frame.</br>
4 (or) cv2.CAP_PROP_FRAME_HEIGHT  =>   Sets the height of the video frame.</br>
5 (or) cv2.CAP_PROP_FPS	          =>   Sets the frames per second (FPS) of the video.</br>

In [2]:
mp_hands = mp.solutions.hands 
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1,min_detection_confidence=0.7,min_tracking_confidence=0.7)

In [3]:
video = cv2.VideoCapture(0)

frame_width = 640
frame_height = 480
video.set(3,frame_width)
video.set(4,frame_height)
canvas = np.zeros((frame_height,frame_width,3),dtype=np.uint8)

prev_x = None
prev_y = None

fontColor = (255,255,255)

while True:
    ret,frame = video.read()
    if not ret:
        break
    
    frame = cv2.flip(frame,1)
    
    #Convert BGR to RGB
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    #Finding hand landmarks
    result = hands.process(image)
    handLandmarks = result.multi_hand_landmarks
    
    #Convert BGR to RGB
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    
    cv2.rectangle(image,(20,10),(152,90),(255,255,255),-1)
    cv2.rectangle(image,(172,10),(304,90),(255,0,0),-1)
    cv2.rectangle(image,(324,10),(456,90),(0,255,0),-1)
    cv2.rectangle(image,(476,10),(608,90),(0,255,255),-1)
    
    cv2.rectangle(canvas,(20,10),(152,90),(255,255,255),-1)
    cv2.rectangle(canvas,(172,10),(304,90),(255,0,0),-1)
    cv2.rectangle(canvas,(324,10),(456,90),(0,255,0),-1)
    cv2.rectangle(canvas,(476,10),(608,90),(0,255,255),-1)
    
    cv2.putText(image,"Clear",(46,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),3)
    cv2.putText(canvas,"Clear",(46,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),3)
    
    #Drawing the landmarks in image
    if handLandmarks:
        for hand_landmark in handLandmarks:
            
            mp_drawing.draw_landmarks(image,hand_landmark,mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec((255, 0, 0),thickness=2,circle_radius=4),
                                      mp_drawing.DrawingSpec((0, 255, 255),thickness=2,circle_radius=4))
            
            index_tip = hand_landmark.landmark[8]
            index_middle = hand_landmark.landmark[6]
            index_x = int(index_tip.x*frame_width)
            index_y = int(index_tip.y*frame_height)
            
            middle_finger_tip = hand_landmark.landmark[12]
            middle_finger_middle = hand_landmark.landmark[10]
            middle_tip_x = int(middle_finger_tip.x*frame_width)
            middle_tip_y = int(middle_finger_tip.y*frame_height)
            middle_midle_y = int(middle_finger_middle.y*frame_height)
            
            if index_y < index_middle.y*frame_height and  middle_tip_y < middle_midle_y:
                    cv2.line(image,(index_x,index_y-15),(middle_tip_x,index_y-20),(0,0,0),20)
                    # canvas = cv2.line(canvas,(prev_x,prev_y),(index_x,index_y),(255,255,0),3)
                    if middle_tip_y < 85:
                        if middle_tip_x > 20 and middle_tip_x < 152:
                            changeShape = canvas[91:,:].shape
                            canvas[91:,:] = np.zeros(changeShape,dtype=np.uint8)
                        elif middle_tip_x > 172 and middle_tip_x < 304:
                            fontColor = (255,0,0)
                        elif middle_tip_x > 324 and middle_tip_x < 456:
                            fontColor = (0,255,0)
                        elif middle_tip_x > 476 and middle_tip_x < 608:
                            fontColor = (0,255,255)
                        
                        prev_x,prev_y = None,None
                        
            elif index_y < index_middle.y*frame_height:
                if index_y > 93:
                    if prev_x is None and prev_y is None:
                        prev_x,prev_y = index_x,index_y
                    else:
                        cv2.line(image,(index_x,index_y),(index_x,index_y),(0,0,255),20)
                        canvas = cv2.line(canvas,(prev_x,prev_y),(index_x,index_y),fontColor,3)
                        prev_x,prev_y = index_x,index_y
            else:
                prev_x,prev_y = None,None
    
    # Merge canvas with frame
    # image = cv2.addWeighted(image, 0.7, canvas, 0.3, 0)
    
    cv2.imshow("Camera",image)
    cv2.imshow("paint",canvas)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
video.release()
cv2.destroyAllWindows()